In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# columns of interest
cols_to_extract = ['imonth', 'iyear','iday',
 'region_txt','country_txt',
 'gname',
 'attacktype1_txt',
 'success','suicide',
 'weaptype1_txt',
 'targtype1_txt',
 'individual','nperps',
 'claimed',
 'nkill','nwound',
 'propextent_txt',
 'ishostkid','hostkidoutcome_txt','ransom']

# Load data
df = pd.read_excel('GTD_0617dist/globalterrorismdb_0617dist.xlsx',
                   usecols=cols_to_extract,
                   na_values = ['Unknown','-99','-9','Not Applicable'])

# replace some values not correctly dealt with by pandas import
df.replace(-9,np.nan,inplace=True)
df.replace(-99,np.nan,inplace=True)

# entries without month or day - treat as 1st January
df['imonth'].replace(0,1,inplace=True)
df['iday'].replace(0,1,inplace=True)

# create a date column, then get rid of the month and day columns
df['date']=pd.to_datetime(dict(year=df.iyear, month=df.imonth, day=df.iday)) 
df.rename(columns={'iyear':'year'}, inplace=True)
df.drop(['imonth','iday'],axis=1,inplace=True)

# set date as the index
df.set_index('date',inplace=True)

display(df.head(5))
df.info()

In [ ]:
# remove events with no group affiliation
no_grp = df.gname.isnull() | df.individual
with_grp = ~no_grp
df = df[with_grp]

# don't need the 'individual' column any more
df.drop('individual',axis=1,inplace=True)

# only keep the top 100 groups with the most incidents
n_groups = 100
top_grps = df['gname'].value_counts().head(n_groups).index
df = df[df.gname.isin(top_grps)]

print('Number of events affiliated with individuals or unknown group: ',sum(no_grp))
print('Number of events affiliated with a group: ',sum(with_grp))
print('Number of events affiliated with top 100 groups: ',len(df))
print('')

df.info()

In [ ]:
# Categorise some columns to help feature extraction later

# Numeric columns - convert to values for 0, 1, 2-10, and more than 10 
conv_numeric = ['nkill','nwound','nperps']

for col in conv_numeric:
    df[col] = pd.cut(df[col],
                        [-0.1,0.9,1.9,10.9,max(df[col])+0.1],
                        labels=['0_'+col,'1_'+col,'2to10_'+col,'11+_'+col])

# boolean columns - add column suffix
conv_bool = ['success','suicide','claimed','property','ishostkid','ransom']

for col in conv_bool:
    df[col].replace({0:('0_'+col),1:('1_'+col)},inplace=True)

df.head()

In [ ]:
# calculate characterising values for each group
# i.e. the feature values that are most characteristic
# of an attack by each group

grp_weights = dict()

# loop over all columns excluding gname
for col in df.drop('gname',axis=1).columns:
    # for each group, how many times each unique value appears in this column
    grp_cnts = df.groupby('gname')[col].value_counts().unstack(col,fill_value=0)
    
    # for each unique value in this column, count how many groups have an incident including it
    # convert this for tf-idf weight using log(n_groups/count)
    w_col = np.log(n_groups/(grp_cnts>0).sum())
    
    # multiply w_col by no. occurences each column value to get weight for each group
    grp_weights[col] = grp_cnts*w_col

    # print some values for the Taliban
    print(grp_weights[col].loc['Taliban'].sort_values(ascending=False).head(2))
    print('----------------------------------------------')
    
#grpw_all = pd.merge(grp_weights['country_txt'],grp_weights['year'],left_index=True,right_index=True)
#
#for grp in top_grps:
#    print(grpw_all.loc[grp].sort_values(ascending=False).head(3))
#    print('----------------------------------------------')